<a href="https://colab.research.google.com/github/grushilkumar/NLP-Text-analytics/blob/main/sentence_comparator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install dask[dataframe]
!python -m pip install nest_asyncio
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from itertools import product
import numpy
import dask
import dask.dataframe as dd
from dask.delayed import delayed
import pandas as pd
import string
import re
from openpyxl import load_workbook
import multiprocessing
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

     |████████████████████████████████| 112kB 12.8MB/s 
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
file_loc = "/content/drive/MyDrive/Service items for SAC Update - Copy.xlsx"
#df2 = dd.read_excel(file_loc, sheet_name="", index_col=None, na_values=['NA'], usecols = "B,D")

part1 = dask.delayed(pd.read_excel)(file_loc, sheet_name="SAC STD", usecols = "D,B,E")#
part1.to_frame(name='SAC Code')
part1.to_frame(name='SAC Description')
part2 = dask.delayed(pd.read_excel)(file_loc, sheet_name="Service items", usecols = "H,I")#, skipfooter = 78040
df1 = dd.from_delayed(part1)
df2 = dd.from_delayed(part2)

In [ ]:
a = []
for i in df1["SAC Description"]:
  print(i)
  i = i.lower()
  a.append(i)

def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list

desc = []
for i in df1["Description"]:
  i = re.sub('nan','',str(i))
  i = i.lower()
  desc.append(i)

resulted = [i + j for i, j in zip(a, desc)]
a = resulted 

b = []
for i in df2["itemdesc"]:
  i = re.sub('SCOPE :- As described further; DETAILS :- SCOPE : -As described further ; DETAILS : - ','',str(i))
  i = re.sub('SCOPE :- As described further; DETAILS :- ','',str(i))
  i = re.sub('SCOPE :- As described further; ','',str(i))
  i = re.sub('SCOPE','',str(i))
  i = re.sub('WORK CATEGORY : Commercial services. APPLICATION : ','',str(i))
  i = re.sub('DIFFERENTIATOR :','', str(i))
  i = re.sub('NATURE OF SERVICE : Community(Exclusive)','', str(i))
  i = re.sub('APPLICATION : Corporate Social Responsibility','', str(i))
  i = re.sub('SCOPE INVOLVES :','', str(i))
  i = re.sub('Labor','labour',str(i))
  # i = re.sub('charges','',str(i))
  # i = re.sub('Charges','',str(i))
  i = i.lower()
  for c in string.punctuation:
    i= i.replace(c," ")
    re.sub(' +', ' ', i) 
  b.append(i)
b=remove(b)

code = []
for i in df1["SAC Code"]:
  code.append(i)

dcode = []
for i in df2["SACcode"]:
  dcode.append(i)


Construction services of single dwelling or multi dewlling or multi-storied residential buildings
Construction services of other residential buildings such as old age homes, homeless shelters, hostels etc
Construction services of industrial buildings such as buildings used for production activities (used for assembly line activities), workshops, storage buildings and other similar industrial buildings
Construction services of commercial buildings such as office buildings, exhibition & marriage halls, malls, hotels, restaurants, airports, rail or road terminals, parking garages, petrol and service stations, theatres and other similar buildings.
Construction services of other non-residential buildings such as educational institutions, hospitals, clinics including vertinary clinics, religious establishments, courts, prisons, museums and other similar buildings
Construction Services of other buildings n.e.c
Services involving Repair, alterations, additions, replacements, renovation, mainte

In [ ]:
def myfunc(sentence1, number):
  def sentence_comparator(sentence1,sentence2):
    str1 = sentence1
    str2 = sentence2

    ##---------------Defining stopwords for English Language---------------##
    stop_words = set(stopwords.words("english"))

    ##---------------Initialising Lists---------------##
    filtered_sentence1 = []
    filtered_sentence2 = []
    lemm_sentence1 = []
    lemm_sentence2 = []
    sims = []
    temp1 = []
    temp2 = []
    simi = []
    final = []
    same_sent1 = []
    same_sent2 = []
    


    ##---------------Defining WordNet Lematizer for English Language---------------##
    lemmatizer  =  WordNetLemmatizer()
    ps = PorterStemmer() 


    ##---------------Tokenizing and removing the Stopwords---------------##

    for words1 in word_tokenize(str1):
        if words1 not in stop_words:
            if words1.isalnum():
                filtered_sentence1.append(words1)

    ##---------------Lemmatizing: Root Words---------------##

    for i in filtered_sentence1:
        lemm_sentence1.append(ps.stem(i))
        
    ##---------------Tokenizing and removing the Stopwords---------------##

    for words2 in word_tokenize(str2):
        if words2 not in stop_words:
            if words2.isalnum():
                filtered_sentence2.append(words2)

    ##---------------Lemmatizing: Root Words---------------##

    for i in filtered_sentence2:
        lemm_sentence2.append(ps.stem(i))


    ##---------------Similarity index calculation for each word---------------##
    for word1 in lemm_sentence1:
        simi =[]
        for word2 in lemm_sentence2:
            sims = []
            syns1 = wordnet.synsets(word1)
            syns2 = wordnet.synsets(word2)
            for sense1, sense2 in product(syns1, syns2):
                d = wordnet.wup_similarity(sense1, sense2)
                if d != None:
                    sims.append(d)
        
            if sims != []:        
              max_sim = max(sims)
              simi.append(max_sim)
                
        if simi != []:
            max_final = max(simi)
            final.append(max_final)


    ##---------------Final Output---------------##

    similarity_index = numpy.mean(final)
    similarity_index = round(similarity_index , 2)
    print("Sentence 1: ",str1)
    #print("Sentence 2: ",str2)
    max_no.append(similarity_index)
    print("Similarity index value : ", similarity_index)
    similarity.append(similarity_index)
  

  max_no =[]
  similarity = []
  SACcode=[]
  #sentence1 = b[number]
  x=1
  N = 25
  #for running the sentence comparator function  
  for i in a:
    print(x)
    x+=1
    sentence_comparator(sentence1,i)

  #finding the max position for each sentence
  maxpos = max_no.index(max(max_no)) 
  print("The maximum is at position", maxpos + 1)

  #creating a list of index with top N sentences with higher similarity
  res = sorted(range(len(max_no)), key = lambda sub: max_no[sub])[-N:]
  for i in range(len(res)):
      res[i] += 1
  print("Indices list of max N elements is : " + str(res)) 

  #creating a list of top N SAC Codes
  for index in res:
    SACcode.append(code[index-1])

  #creating a list of top N Sentences
  result = []
  for i in res:
    result.append(a[i-1])
  print(result)

  #adding the  content into the file
  f=open('f1.txt','w')
  for ele in result:
    ele = ele.lower()
    for c in string.punctuation:
      ele= ele.replace(c," ")
      re.sub(' +', ' ', ele)  
    f.write(str(ele)+'.\n')
  f.close()

  f=open('f2.txt','w')
  f.write(sentence1)
  f.close()

  file_docs = []
  with open ('f1.txt') as f:
      tokens = sent_tokenize(f.read())
      for line in tokens:
          file_docs.append(line)
  print("Number of documents:",len(file_docs))
  
  gen_docs = [[w.lower() for w in word_tokenize(text)] for text in file_docs]
  
  import gensim
  from gensim.parsing.preprocessing import remove_stopwords
  dictionary = gensim.corpora.Dictionary(gen_docs)
  
  corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
  
  import numpy as np
  tf_idf = gensim.models.TfidfModel(corpus)

  sims = gensim.similarities.Similarity('/content/sample_data/',tf_idf[corpus],num_features=len(dictionary))

  file2_docs = []

  with open ('f2.txt') as f:
      tokens = sent_tokenize(f.read())
      for line in tokens:
          file2_docs.append(line)

  print("Number of documents:",len(file2_docs))  
  for line in file2_docs:
      query_doc = [w.lower() for w in word_tokenize(line)]
      query_doc_bow = dictionary.doc2bow(query_doc)

  query_doc_tf_idf = tf_idf[query_doc_bow]

  avg_sims = [] # array of averages
  max_sim=[]
  index_sim = []
  # for line in query documents
  for line in file2_docs:
          # tokenize words
          query_doc = [w.lower() for w in word_tokenize(line)]
          # create bag of words
          query_doc_bow = dictionary.doc2bow(query_doc)
          # find similarity for each document
          query_doc_tf_idf = tf_idf[query_doc_bow]
          # print (document_number, document_similarity)
          print('Comparing Result:', sims[query_doc_tf_idf]) 
          #printing max similar attribute
          max_sim.append(max(sims[query_doc_tf_idf]))
          index_sim.append(np.argmax(sims[query_doc_tf_idf])+1)
          print('max similarity:', max(sims[query_doc_tf_idf]), 'index:', np.argmax(sims[query_doc_tf_idf]))
          required_sentence = result[np.argmax(sims[query_doc_tf_idf])]
          required_index = res[np.argmax(sims[query_doc_tf_idf])]
          required_SACcode = SACcode[np.argmax(sims[query_doc_tf_idf])]
          print(result[np.argmax(sims[query_doc_tf_idf])])
          sent = file_docs[np.argmax(sims[query_doc_tf_idf])]
          # calculate sum of similarities for each query doc
          sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
          # calculate average of similarity for each query doc
          avg = sum_of_sims / len(file_docs)
          # print average of similarity for each query doc
          print(f'avg: {sum_of_sims / len(file_docs)}')
          # add average values into array
          avg_sims.append(avg)  
          # calculate total average
          total_avg = np.sum(avg_sims, dtype=np.float)
          # round the value and multiply by 100 to format it as percentage
          percentage_of_similarity = round(float(total_avg) * 100)
          # if percentage is greater than 100
          # that means documents are almost same
          if percentage_of_similarity >= 100:
            percentage_of_similarity = 100

  print(required_sentence)
  print(required_index)
  print(sentence1)
  workbook = load_workbook(file_loc)
  sheet = workbook.active
  sheet.cell(row=number+2, column= 9 ).value = required_SACcode
  workbook.save(file_loc)




In [ ]:
if __name__ == "__main__":
  #number = number-2
  number = 0
#mention the range of the items description
  b_compact = b[0:3]
  for i in b_compact:
    myfunc(i,number)
    number+=1

1
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.62
2
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.68
3
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.77
4
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.77
5
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.78
6
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.47
7
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.64
8
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.88
9
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.83
10
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.8
11
Sentence 1:  details    l c for doing back filling of soi

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.67
130
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.65
131
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.7
132
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.4
133
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.83
134
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.7
135
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.9
136
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.81
137
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.54
138
Sentence 1:  details    l c for doing back filling of soil  
Similarity index value :  0.6
139
Sentence 1:  details    l c for doing back fi

/usr/local/lib/python3.7/dist-packages/gensim/similarities/docsim.py:518: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  result = numpy.hstack(shard_results)
